In [1]:
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
import json

In [2]:
base_payload_str = """{"pastWetr":{"naverRgnCd":"09110178","aplYm":""}}"""
base_payload = json.loads(base_payload_str)  
aplYm_list = []
for year in range(2020, 2025):
    for month in range(1, 13):
        aplYm = f"{year}{month:02d}"  
        if int(aplYm) <= 202405:  
            base_payload["pastWetr"]["aplYm"] = aplYm
            aplYm_list.append(json.dumps(base_payload))  

In [3]:
all_df = pd.DataFrame()
for i in tqdm(range(len(aplYm_list))):
    url = ""
    try:
        r = requests.get(url)
        past_wetr_list = [data['pastWetr'] for data in r.json()['results']['choiceResult']['pastWetr']['pastWetrCalendarList']]
        df = pd.DataFrame(past_wetr_list)
        all_df = pd.concat([all_df,df])
    except:
        print(aplYm_list[i])
    

 66%|██████████████████████████████████████████████████████▊                            | 35/53 [00:04<00:02,  8.70it/s]

{"pastWetr": {"naverRgnCd": "09110178", "aplYm": "202209"}}


100%|███████████████████████████████████████████████████████████████████████████████████| 53/53 [00:06<00:00,  7.78it/s]

{"pastWetr": {"naverRgnCd": "09110178", "aplYm": "202405"}}


# 아래 2케이스의 값 수기로 받아오기
{"pastWetr": {"naverRgnCd": "09140104", "aplYm": "202209"}}
{"pastWetr": {"naverRgnCd": "09140104", "aplYm": "202405"}}

In [4]:
add_payload = """{"pastWetr":{"naverRgnCd":"09110178","aplYm":"202209"}}"""
add_url = ""
add_r = requests.get(add_url)
add_past_wetr_list = [data['pastWetr'] for data in add_r.json()['results']['choiceResult']['pastWetr']['pastWetrCalendarList']]
# none값 오류뜸
add_df = pd.DataFrame([{k: v for k, v in data.items() if k != 'reportYmdt'} for data in add_past_wetr_list  if data])

In [5]:
add_payload_1 = """{"pastWetr":{"naverRgnCd":"09110178","aplYm":"202405"}}"""
add_url_1 = ""
add_r_1 = requests.get(add_url_1)
add_past_wetr_list_1 = [data['pastWetr'] for data in add_r_1.json()['results']['choiceResult']['pastWetr']['pastWetrCalendarList']]
# none값 오류뜸 + data 없는값 존재
add_df_1 = pd.DataFrame([{k: v for k, v in data.items() if k != 'reportYmdt'} for data in add_past_wetr_list_1 if data])

In [6]:
all_df.columns

Index([    'aplYmd',    'lareaNm',    'mareaNm',     'wetrCd',    'wetrTxt',
          'minTmpr',    'maxTmpr',    'rainAmt', 'reportYmdt',            0],
      dtype='object')

In [7]:
add_df.columns

Index(['aplYmd', 'lareaNm', 'mareaNm', 'wetrCd', 'wetrTxt', 'minTmpr',
       'maxTmpr', 'rainAmt'],
      dtype='object')

In [8]:
add_df_1.columns

Index(['aplYmd', 'lareaNm', 'mareaNm', 'wetrCd', 'wetrTxt', 'minTmpr',
       'maxTmpr', 'rainAmt'],
      dtype='object')

In [9]:
# all_df add_df add_df_1 데이터 통함
all_df.drop(columns=["minTmpr", "maxTmpr", "rainAmt", "reportYmdt", 0], axis=1, inplace=True)
add_df.drop(columns=["minTmpr", "maxTmpr", "rainAmt"], axis=1, inplace=True)
add_df_1.drop(columns=["minTmpr", "maxTmpr", "rainAmt"], axis=1, inplace=True)

In [10]:
display(all_df.head(1))
display(add_df.head(1))
display(add_df_1.head(1))

,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
0,20191229,서울특별시,종로구,9,흐림/비


,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
0,20220828,서울특별시,종로구,3,구름조금


,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
0,20240428,서울특별시,종로구,5,구름많음


aplYmd 날짜
lareaNm 서울특별시
mareaNm 중구
wetrCd 날씨코드
wetrTxt 날씨설명!
minTmpr 최저온도	
maxTmpr 최고온도
rainAmt 강수

In [11]:
new_all_df = pd.concat([all_df,add_df,add_df_1])

In [12]:
new_all_df[new_all_df["aplYmd"] == "20200126"]

,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
28,20200126,서울특별시,종로구,17,구름많음/안개
0,20200126,서울특별시,종로구,17,구름많음/안개


In [13]:
print(new_all_df[new_all_df.duplicated(subset=['aplYmd'], keep=False)]["aplYmd"])
# 20200126

28    20200126
29    20200127
30    20200128
31    20200129
32    20200130
        ...   
2     20240430
3     20240501
4     20240502
5     20240503
6     20240504
Name: aplYmd, Length: 658, dtype: object


In [14]:
new_all_df.drop_duplicates(subset=['aplYmd'], keep='first', inplace=True) 

In [15]:
new_all_df[new_all_df["aplYmd"] == "20200126"]

,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
28,20200126,서울특별시,종로구,17,구름많음/안개


In [16]:
new_all_df = new_all_df.sort_values(by='aplYmd')

In [17]:
new_all_df.to_csv('new_data.csv', index=False, encoding='utf-8-sig')